In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy.sparse import csc_matrix, csr_matrix, hstack
import time
import numpy as np
import pickle

from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization,CuDNNLSTM,Activation,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.utils import shuffle
from keras.preprocessing import text, sequence
from sklearn.metrics import log_loss,roc_auc_score
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
import re
from keras import backend as K
import time
import gc
# To get learning rate


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
train = pd.read_pickle(FILE.train_ori.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_ori.value)
print('test shape is: {}'.format(test.shape))
train_length = len(train)

X_shiyi = pd.read_pickle(FILE.shiyi_fillna_ori.value)
# X_shiyi['weekDay'] = X_shiyi['time_day'] % 7 +1
print(X_shiyi.shape)

train = train.merge(X_shiyi[['time_hour','instance_id']],how='inner',on='instance_id')
test = test.merge(X_shiyi[['time_hour','instance_id']],how='inner',on='instance_id')


X = pd.concat([train,test],sort=False)
print(X.shape)
X = X.reset_index(drop=True)

# X_clean = pd.read_csv('../../data/original/cleaned_data_price_final.csv')
# X_append = pd.read_pickle('../../data/nn_features/clean.pkl')



print(X.shape)
# assert np.sum(X_clean['instance_id'].values != X['instance_id'].values) == 0
# assert np.sum(X_append['instance_id'].values != X['instance_id'].values) == 0
# X['model'] = X_append['model_new'].values


ignore_columns = ['instance_id','time','click'] + ['creative_is_js', 'creative_is_voicead', 'app_paid']

train shape is: (1001650, 35)
test shape is: (40024, 34)
(1041674, 229)
(1041674, 36)
(1041674, 36)


# PB 4236

In [3]:
# need_process_col = list(set(X.columns) - set(ignore_columns))
# X_ = X[need_process_col].copy()


# doc_col = ['user_tags','model']
# non_doc_col = [f for f in need_process_col if f not in doc_col]
# # doc_col = doc_col + []
# # non_doc_col = non_doc_col+['user_tags']
# counter = 0

# X_doc = X[doc_col].copy()
# for col in tqdm(non_doc_col):
#     X_[col] = le.fit_transform(X_[col].astype(str))
#     X_[col] = col + '_'+X_[col].astype(str)
    
# for col in tqdm(doc_col):
#     X_doc[col] = X_doc[col].astype(str)
    
    

    

In [4]:
need_process_col = list(set(X.columns) - set(ignore_columns))
X_ = X[need_process_col].copy()


doc_col = ['user_tags','model']
non_doc_col = [f for f in need_process_col if f not in doc_col]
# doc_col = doc_col + []
# non_doc_col = non_doc_col+['user_tags']
counter = 0

X_doc = X[doc_col].copy()
for col in tqdm(non_doc_col):
#     col = 'creative_id'
    test_values = set(X_[col].iloc[train_length:].astype(str).unique())
    train_values = set(X_[col].iloc[:train_length].astype(str).unique())
    intersection = train_values.intersection(test_values)
    out_liyer = list(train_values.union(test_values) - intersection)
    if len(out_liyer) > 0:
#         print(len(out_liyer))
        out_liyer_mapping = pd.Series(index=list(out_liyer),data=1)
        filtered = (X_[col].astype(str).map(out_liyer_mapping) == 1)
#         print('col:{}, size:{}'.format(col,np.sum(filtered)))
        X_.loc[filtered,col] = np.nan
        
        
    X_[col] = le.fit_transform(X_[col].astype(str))
    X_[col] = col + '_'+X_[col].astype(str)
    
for col in tqdm(doc_col):
    X_doc[col] = X_doc[col].astype(str)
    
    

    

100%|██████████| 2/2 [00:00<00:00,  6.47it/s]


In [5]:
def get_tok(X,col,train_length,d_filter='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'):
    X[col] = X[col].astype(str)

    tok_all=text.Tokenizer(num_words=X[col].nunique(),lower=False,filters=d_filter)
    tok_all.fit_on_texts(list(X[col].values))

    tok_train=text.Tokenizer(num_words=X[col].iloc[:train_length].nunique(),lower=False,filters=d_filter)
    tok_train.fit_on_texts(list(X[col].iloc[:train_length].values))

    tok_test=text.Tokenizer(num_words=X[col].iloc[train_length:].nunique(),lower=False,filters=d_filter)
    tok_test.fit_on_texts(list(X[col].iloc[train_length:].values))
    word_intersection = set(tok_train.word_index.keys()).intersection(set(tok_test.word_index.keys()))

    self_index = {}
    count = 1
    for word in word_intersection:
        self_index[word] = count
        count+=1
    self_index['unknown'] = count
    print('max index is: {}'.format(count))

    for word in tok_all.word_index.keys():
        tok_all.word_index[word] = self_index.get(word,count)
    return tok_all

# Combine columns

In [6]:
# # def comb_fe(X,cols,sep=' '):
# #     ret = X[cols[0]].astype(str).copy()
# #     for col in cols[1:]:
# #         ret = ret + sep + X[col].astype(str)
# #     return ret.values

# def process_slot(x):
#     x = re.sub(r'[-_]',' ',x)
#     x = x.split(' ')
#     ret = ['p{}_'.format(c)+x[c] for c in range(len(x))]
#     ret = ' '.join(ret)
#     return ret

# def comb_fe(X,cols,sep=' '):
#     def add_col_name(x,colName,splitter=' ' ,sep=' '):
#         ret = [colName+'_'+each for each in x.split(splitter)]
#         return sep.join(ret)
#     ret = pd.DataFrame()
#     if cols[0] == 'user_tags':
#         ret['comb'] = X['user_tags'].astype(str).apply(add_col_name,colName='userTags',splitter=',',sep=sep)
#     else:
#         if cols[0] in original_name_col:
#             ret['comb'] = X[cols[0]].astype(str).copy()
#         else:
#             ret['comb'] = X[cols[0]].astype(str).apply(add_col_name,colName=cols[0],splitter=' ',sep=sep)

#     for col in tqdm(cols[1:]):
#         if col == 'user_tags':
#             ret['new_feature'] = X['user_tags'].astype(str).apply(add_col_name,colName='userTags',splitter=',',sep=sep)
#         else:
#             if col in original_name_col:
#                 ret['new_feature'] = X[col].astype(str).copy()
#             else:
#                 ret['new_feature'] = X[col].astype(str).apply(add_col_name,colName=col,splitter=' ',sep=sep)
#         ret['comb'] =( ret['comb'] + sep + ret['new_feature']).values
#     return ret['comb'].values

# processed_col = []
# original_name_col = ['model','make','os','osv']
# # cob_col = [['model','make','os','osv']]
# # cob_col = [['model','make','os','osv'],['model','app_id','inner_slot_id','creative_width', 'creative_height']]
# # cob_col = [['model', 'osv']]
# cob_col=[]
# doc_col=['user_tags','model']
# # doc_col=['user_tags']
# X_doc = X[doc_col].copy()
# processed_col.extend(doc_col)
# for each in cob_col:
#     feature_name = '_'.join(each)
#     processed_col.extend(each)
#     doc_col.append(feature_name)
#     X_doc[feature_name] = comb_fe(X,each)
    
# non_doc_col = list(set(X.columns) - set(ignore_columns) - set(processed_col))
# X_ = X[non_doc_col].copy()

# for col in tqdm(non_doc_col):
#     X_[col] = le.fit_transform(X_[col].astype(str))
#     X_[col] = col + '_'+X_[col].astype(str)
    
# for col in tqdm(doc_col):
#     if col=='inner_slot_id':
#         X_doc[col] = X_doc[col].astype(str).apply(process_slot)
#     else:
#         X_doc[col] = X_doc[col].astype(str)

In [7]:
# train_index = pickle.load(open(FILE.train_index.value,'rb'))
# holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))

# train_index = pickle.load(open('../../data/original/adversarial_train_index.pickle','rb'))
# holdout_index = pickle.load(open('../../data/original/adversarial_ho_index.pickle','rb'))

In [8]:
train_index = train.index
holdout_index = None

In [9]:
num_folds = 5
seed = 1001
train_index_list = []
val_index_list = []
folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
for t,v in folds.split(train.loc[train_index],train.loc[train_index,'click']):
    train_index_list.append(train.loc[train_index].iloc[t].index)
    val_index_list.append(train.loc[train_index].iloc[v].index)
    
# check = []
# for i in val_index_list:
#     check.extend(list(i))
# assert len(set(check+list(holdout_index))) == len(train)

In [10]:
train_fold_y = {}
val_fold_y = {}
if holdout_index is not None:
    holdout_y = train.loc[holdout_index,'click'].values
else:
    holdout_y = None
for fold in range(num_folds):
    train_fold_y[fold] = train.loc[train_index_list[fold],'click'].values
    val_fold_y[fold] = train.loc[val_index_list[fold],'click'].values

In [11]:
info_dict = {}
train_all_dict = {}
train_fold_dict = {}

val_fold_dict = {}

holdout_input_dict = {}
test_input_dict = {}
maxlen = 1
prefix_input_nonDoc = 'input_'
prefix_input_Doc = 'input_rnn_'
for col in tqdm(non_doc_col):

    maxlen = 1
    tok=text.Tokenizer(num_words=X_[col].nunique(),lower=False,filters='@')
    tok.fit_on_texts(list(X_[col]))
    info_dict.update({prefix_input_nonDoc+col:{'tok':tok}})
    t = tok.texts_to_sequences(list(X_[col].iloc[:train_length].values))
    te = tok.texts_to_sequences(list(X_[col].iloc[train_length:].values))
    train_all_dict[prefix_input_nonDoc+col] = sequence.pad_sequences(t,maxlen=maxlen)
    test_input_dict[prefix_input_nonDoc+col] = sequence.pad_sequences(te,maxlen=maxlen)
    if holdout_index is not None:
        holdout_input_dict[prefix_input_nonDoc+col] = train_all_dict[prefix_input_nonDoc+col][list(holdout_index)]
    
    for fold in range(num_folds):
        if train_fold_dict.get(fold) is None:
            train_fold_dict[fold] = {}
            val_fold_dict[fold] = {}
        train_fold_dict[fold].update({prefix_input_nonDoc+col: train_all_dict[prefix_input_nonDoc+col][list(train_index_list[fold])]})
        val_fold_dict[fold].update({prefix_input_nonDoc+col:train_all_dict[prefix_input_nonDoc+col][list(val_index_list[fold])]})
        
sequence_size_dict = {}
for col in tqdm(doc_col):
    if col == 'user_tags':
        maxlen = 50
        tok = get_tok(X_doc,col=col,train_length=train_length,d_filter=',')
#         tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False, filters=',')
#         tok.fit_on_texts(list(X_doc[col]))

        
    elif col == 'model':
        maxlen = 15
#         tok = get_tok(X_doc,col=col,train_length=train_length)
        tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False)
        tok.fit_on_texts(list(X_doc[col]))
    else:
        maxlen = 15
#         tok = get_tok(X_doc,col=col,train_length=train_length)
        tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False)
        tok.fit_on_texts(list(X_doc[col]))
    info_dict.update({prefix_input_Doc+col:{'tok':tok}})
    sequence_size_dict[col] = maxlen
    t = tok.texts_to_sequences(list(X_doc[col].iloc[:train_length].values))
    te = tok.texts_to_sequences(list(X_doc[col].iloc[train_length:].values))
    train_all_dict[prefix_input_Doc+col] = sequence.pad_sequences(t,maxlen=maxlen)
    test_input_dict[prefix_input_Doc+col] = sequence.pad_sequences(te,maxlen=maxlen)
    if holdout_index is not None:
        holdout_input_dict[prefix_input_Doc+col] = train_all_dict[prefix_input_Doc+col][list(holdout_index)]
    
    for fold in range(num_folds):
        if train_fold_dict.get(fold) is None:
            train_fold_dict[fold] = {}
            val_fold_dict[fold] = {}
        train_fold_dict[fold].update({prefix_input_Doc+col: train_all_dict[prefix_input_Doc+col][list(train_index_list[fold])]})
        val_fold_dict[fold].update({prefix_input_Doc+col:train_all_dict[prefix_input_Doc+col][list(val_index_list[fold])]})

  0%|          | 0/2 [00:00<?, ?it/s]

max index is: 1313


100%|██████████| 2/2 [00:56<00:00, 28.42s/it]


In [12]:
tok = info_dict['input_rnn_model']['tok']

In [13]:
tok.word_index

{'25221605': 7354,
 '252525252525252BY71': 4068,
 '���6s': 6973,
 'L168': 4101,
 'D6616': 6507,
 '25252525252BA33': 1807,
 '25252BD820u': 7265,
 'D22AP': 6320,
 '25252B509': 8026,
 '20Xplay5A': 1135,
 'N2': 1694,
 'C8818': 767,
 '25252BY27': 4425,
 '2525252BA73t': 2661,
 'C9': 1725,
 '2525252525252BGN5007L': 5197,
 '252525252525252BR11': 925,
 'G3226': 6258,
 'M040': 1869,
 '1841': 5518,
 'metal': 183,
 'V3Max': 68,
 'T708S': 3735,
 '20160212w': 8666,
 '252525252525252BA53m': 3807,
 '252525252B5891Q': 7679,
 'CPH1607': 2994,
 '25252BNote': 598,
 '25252525252BX6L': 4702,
 '25252BA83t': 1740,
 'G9650': 527,
 'N9009': 542,
 '1011T': 5666,
 'G1650': 3870,
 'OPPOA33': 1436,
 '2522EVA': 8449,
 '2BC8818': 8568,
 '2B3X': 7646,
 'OPPOA33t': 5781,
 '8712S': 1786,
 'TbooK': 8497,
 '2525252525252BY913': 9232,
 '252525252525252BX6SPlus': 2979,
 'x20plus': 2337,
 'N5': 1319,
 '25252BSC': 6083,
 'E5823': 4353,
 'h1s': 7072,
 'KOPOL8': 8011,
 '252525252BY67L': 2257,
 'VOVI': 8560,
 '25252BMax': 5323,


# Build NN model only use model

# 0.41479

In [14]:
def get_nn_model(cols,doc_cols=[],numu_cols=[]):
    """
    cols, used to do ebd and dense layers
    doc_cols: used to do rnn
    there can be overlaps
    """
    input_list = []
    concat_list = []
    numu_list = []
    for col in cols:
        max_feature = len(info_dict[prefix_input_nonDoc+col]['tok'].index_word)
#         max_feature = len(set(info_dict[prefix_input_nonDoc+col]['tok'].index_word.values()))
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        if embed_size< 2:
            embed_size = 2
        cur_input = Input(shape=(1, ),name = prefix_input_nonDoc+col)
        
       
        embed_layer = Embedding(max_feature,
                            embed_size,
                            input_length=1,
                            trainable=True,
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_'+col)(cur_input)
        embed_layer = SpatialDropout1D(0.5)(embed_layer)
        x = Flatten()(embed_layer)
        input_list.append(cur_input)
        concat_list.append(x)
    for col in doc_cols:
        max_feature = len(info_dict[prefix_input_Doc+col]['tok'].index_word)
#         max_feature = len(set(info_dict[prefix_input_Doc+col]['tok'].index_word.values()))
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        if embed_size< 2:
            embed_size = 2
        input_shape = sequence_size_dict[col]
        cur_input = Input(shape=(input_shape, ),name = prefix_input_Doc+col)
        embed_layer = Embedding(max_feature,
                            embed_size,
                            input_length=input_shape,
                            trainable=True,
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_rnn_'+col)(cur_input)
        x = SpatialDropout1D(0.5)(embed_layer)
        x = Bidirectional(CuDNNGRU(25, return_sequences=True))(x)
        x = Conv1D(25, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
        x_aveP = GlobalAveragePooling1D()(x)
        x_maxP = GlobalMaxPooling1D()(x)
        x = concatenate([x_aveP,x_maxP])
        input_list.append(cur_input)
        concat_list.append(x)

    if len(numu_cols) > 0:
        print('add numu...')
        nu_shape = len(numu_cols)
        cur_input = Input(shape=(nu_shape, ),name = prefix_input_nu)
        x = BatchNormalization()(cur_input)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        input_list.append(cur_input)
        numu_list.append(x)
       
    if len(concat_list) > 1:
        x = concatenate(concat_list)
#     x = BatchNormalization()(x)
    

    if len(numu_list)>0:
        x = concatenate([x]+numu_list)
#         x = BatchNormalization()(x)
        
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)

    preds = Dense(1, activation="sigmoid")(x)
    model = Model(input_list, preds)
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
    return model


In [15]:

# def get_nn_model(cols,doc_cols=[],numu_cols=[]):
#     """
#     cols, used to do ebd and dense layers
#     doc_cols: used to do rnn
#     there can be overlaps
#     """
#     input_list = []
#     concat_list = []
#     numu_list = []
#     for col in cols:
#         max_feature = len(info_dict[prefix_input_nonDoc+col]['tok'].index_word)
#         embed_size = int(np.log2(max_feature)/np.log2(1.5))
#         if embed_size< 2:
#             embed_size = 2
#         cur_input = Input(shape=(1, ),name = prefix_input_nonDoc+col)
        
       
#         embed_layer = Embedding(max_feature,
#                             embed_size,
#                             input_length=1,
#                             trainable=True,
#                             embeddings_regularizer=l2(0.0005),
#                             name='ebd_'+col)(cur_input)
#         embed_layer = SpatialDropout1D(0.5)(embed_layer)
#         x = Flatten()(embed_layer)
#         input_list.append(cur_input)
#         concat_list.append(x)
#     for col in doc_cols:
#         max_feature = len(info_dict[prefix_input_Doc+col]['tok'].index_word)
#         embed_size = int(np.log2(max_feature)/np.log2(1.5))
#         if embed_size< 2:
#             embed_size = 2
#         input_shape = sequence_size_dict[col]
#         cur_input = Input(shape=(input_shape, ),name = prefix_input_Doc+col)
#         embed_layer = Embedding(max_feature,
#                             embed_size,
#                             input_length=input_shape,
#                             trainable=True,
#                             embeddings_regularizer=l2(0.0005),
#                             name='ebd_rnn_'+col)(cur_input)
#         x = SpatialDropout1D(0.5)(embed_layer)
#         x = Bidirectional(CuDNNGRU(256, return_sequences=True))(x)
#         x = Conv1D(128, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
#         x_aveP = GlobalAveragePooling1D()(x)
#         x_maxP = GlobalMaxPooling1D()(x)
#         x = concatenate([x_aveP, x_maxP]) 
#         input_list.append(cur_input)
#         concat_list.append(x)
# #         concat_list.append(x)
#     if len(numu_cols) > 0:
#         print('add numu...')
#         nu_shape = len(numu_cols)
#         cur_input = Input(shape=(nu_shape, ),name = prefix_input_nu)
#         x = BatchNormalization()(cur_input)
#         x = Dense(128, activation='relu')(x)
#         x = Dropout(0.2)(x)
#         input_list.append(cur_input)
#         numu_list.append(x)
       
#     if len(concat_list) > 1:
#         x = concatenate(concat_list)
# #     x = BatchNormalization()(x)
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.2)(x)

#     if len(numu_list)>0:
#         x = concatenate([x]+numu_list)
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.2)(x)

#     preds = Dense(1, activation="sigmoid")(x)
#     model = Model(input_list, preds)
#     model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
#     return model

In [16]:
def train_each_fold(input_train_dict,input_val_dict,y_train,y_val,cols,doc_col=[],tolerance=30,train_batch_size=5000):
    model = get_nn_model(cols,doc_col)
    cur_to = 0
    best_logloss = None
    best_weights = None
    count = 0
    base_lr = 0.001
    while True:
        model.fit(input_train_dict, y_train, 
                  batch_size=train_batch_size, 
                  epochs=1,
                  verbose=1,
                  shuffle=True,
                  )
        preds = model.predict(input_val_dict,5000,verbose=1)
        logloss = log_loss(y_val,preds)
        roc = roc_auc_score(y_val,preds)
        print(logloss)
        print(roc)
        if best_logloss is None:
            best_logloss = logloss
            best_weights = model.get_weights()
        else:
            if best_logloss > logloss:
                best_logloss = logloss
                best_weights = model.get_weights()
                cur_to = 0
            else:
                cur_to +=1
        if cur_to == tolerance:
            break
        print('best logloss is: {}'.format(best_logloss))
        print('remainning trial is: {}/{}'.format(cur_to,tolerance))
        print('total epoch trained: {}'.format(count))
        count+=1
    model.set_weights(best_weights)
    return model
    

In [17]:
def nn_K_fold(train_fold_dict,
              val_fold_dict,
              train_fold_y,
              val_fold_y,
              test_input_dict,
              val_index_list,
              train_df,
              test_df,
              pred_col_name = 'predicted_score',
              holdout_input_dict=None,
              holdout_y=None,
              holdout_index_list=None,
              nondoc_cols=[],
              doc_cols=[],
              tolerance=30,
              train_batch_size=5000,
              preds_batch=5000):
    """
    train_fold_dict: format, key - foldNum, value - nn input 
    train_fold_y: label for train fold, fotmat, key -foldNum, value - label
    val_fold_dict: format, key - foldNum, value - nn input 
    val_fold_y: label for val fold, fotmat, key -foldNum, value - label
    test_input_dict: format, key - foldNum, value - nn input 
    holdout_input_dict: Noneable, if none, not using holdout
    holdout_y: Noneable, label for holdout
    cols: all cols used to do ebd
    doc_cols, cols that are used to do rnn
    val_index_list: cv, going to predict and generate oof
    holdout_index_list: going to predict on each fold
    train_df: 'dataframe which only has id columns, which will be used to store oof prediction'
    test_df: 'dataframe which only has id columns, which will be used to store test prediction'
    """
    train_df = train_df.copy()
    test_df = test_df.copy()
    n_fold = len(train_fold_dict)
    test_preds_list = []
    val_score_list = []
    hold_out_preds_list = []
    holdout_score_list = []
    train_score_list = []
    cv_score_list = []
    train_df[pred_col_name] = np.nan
    test_df[pred_col_name] = np.nan
    
    for fold in range(n_fold):
        print('start fold {}...'.format(fold))
        model = train_each_fold(train_fold_dict[fold],
                                val_fold_dict[fold],
                                train_fold_y[fold],
                                val_fold_y[fold],
                                nondoc_cols,
                                doc_cols,
                                tolerance=tolerance,
                                train_batch_size=train_batch_size)
        train_preds =  model.predict(train_fold_dict[fold],preds_batch,verbose=1)
        val_preds = model.predict(val_fold_dict[fold],preds_batch,verbose=1)
        train_loss = log_loss(train_fold_y[fold],train_preds)
        val_loss = log_loss(val_fold_y[fold],val_preds)
        train_df.loc[val_index_list[fold],pred_col_name] = val_preds
        test_preds = model.predict(test_input_dict,preds_batch,verbose=1)
        test_preds_list.append(test_preds)
        val_score_list.append(val_loss)
        train_score_list.append(train_loss)
        print('Fold {} finish! val loss: {}.'.format(fold,val_loss))
        if holdout_index_list is not None:
            ho_preds = model.predict(holdout_input_dict,preds_batch,verbose=1)
            ho_loss = log_loss(holdout_y,ho_preds)
            ho_roc = roc_auc_score(holdout_y,ho_preds)
            holdout_score_list.append(ho_loss)
            hold_out_preds_list.append(ho_preds)
            print('hold out loss: {}'.format(ho_loss))
        del model
        gc.collect()
        time.sleep(5)
            
    print('finish training... calculating evl matrix')
    test_preds_list = np.array(test_preds_list)
    hold_out_preds_list = np.array(hold_out_preds_list)
    test_preds_final = np.mean(test_preds_list,axis=0)
    cv_score_mean = np.mean(val_score_list)
    train_score_mean = np.mean(train_score_list)
    test_df[pred_col_name] = test_preds_final
    print('cv mean is: {}'.format(cv_score_mean))
    if holdout_index_list is not None:
        ho_preds_final = np.mean(hold_out_preds_list,axis=0)
        ho_loss_overall = log_loss(holdout_y,ho_preds_final)
        ho_roc_overall = roc_auc_score(holdout_y,ho_preds_final)
        train_df.loc[holdout_index_list,pred_col_name] = ho_preds_final
        print('holdout loss overall is: {}'.format(ho_loss_overall))
        print('holdout roc overall is: {}'.format(ho_roc_overall))
        return train_df,test_df,cv_score_mean,ho_loss_overall,train_score_mean
    else:
        return train_df,test_df,cv_score_mean,0.0,train_score_mean
    
        

In [18]:
train_df = train[['instance_id']].copy()
test_df = test[['instance_id']].copy()

train_save,test_save,cv_,ho_,ta_ = nn_K_fold(train_fold_dict,
                                      val_fold_dict,
                                      train_fold_y,
                                      val_fold_y,
                                      test_input_dict,
                                      val_index_list,
                                      train_df,
                                      test_df,
                                      pred_col_name = 'predicted_score',
                                      holdout_input_dict=holdout_input_dict,
                                      holdout_y=holdout_y,
                                      holdout_index_list=holdout_index,
                                      nondoc_cols=non_doc_col,
                                      doc_cols=doc_col,
                                      tolerance=30,
                                      preds_batch=5000)

start fold 0...
Epoch 1/1
200331/200331 [==============================] - 2s 9us/step
0.4216127023342312
0.7571486323693267
best logloss is: 0.4216127023342312
remainning trial is: 0/30
total epoch trained: 0
Epoch 1/1
200331/200331 [==============================] - 2s 8us/step
0.419802271632357
0.7633595199885966
best logloss is: 0.419802271632357
remainning trial is: 0/30
total epoch trained: 1
Epoch 1/1
200331/200331 [==============================] - 2s 7us/step
0.41812591585220116
0.7635814132984957
best logloss is: 0.41812591585220116
remainning trial is: 0/30
total epoch trained: 2
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.41767168856448994
0.7647710166877291
best logloss is: 0.41767168856448994
remainning trial is: 0/30
total epoch trained: 3
Epoch 1/1
200331/200331 [==============================] - 2s 8us/step
0.4175544883726108
0.7652323344674181
best logloss is: 0.4175544883726108
remainning trial is: 0/30
total epoch trained: 4
Epoch 1/1
20

200331/200331 [==============================] - 1s 7us/step
0.41490891917731415
0.7695761105605945
best logloss is: 0.41488552108497345
remainning trial is: 14/30
total epoch trained: 57
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.41536112769721373
0.7696341909862986
best logloss is: 0.41488552108497345
remainning trial is: 15/30
total epoch trained: 58
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.4153742432284854
0.7695616981833966
best logloss is: 0.41488552108497345
remainning trial is: 16/30
total epoch trained: 59
Epoch 1/1
200331/200331 [==============================] - 2s 8us/step
0.41495081335872575
0.7696994064468365
best logloss is: 0.41488552108497345
remainning trial is: 17/30
total epoch trained: 60
Epoch 1/1
200331/200331 [==============================] - 2s 8us/step
0.41520740630979264
0.7694388370517495
best logloss is: 0.41488552108497345
remainning trial is: 18/30
total epoch trained: 61
Epoch 1/1
200331/2003

200331/200331 [==============================] - 1s 7us/step
0.41553236488455964
0.7698605142348403
best logloss is: 0.41553236488455964
remainning trial is: 0/30
total epoch trained: 14
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.4155776676219374
0.7695103393409848
best logloss is: 0.41553236488455964
remainning trial is: 1/30
total epoch trained: 15
Epoch 1/1
200331/200331 [==============================] - 2s 8us/step
0.4157886488776397
0.7698281189477055
best logloss is: 0.41553236488455964
remainning trial is: 2/30
total epoch trained: 16
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.41523735712429555
0.76987875497438
best logloss is: 0.41523735712429555
remainning trial is: 0/30
total epoch trained: 17
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.4150707835067744
0.7704585554816664
best logloss is: 0.4150707835067744
remainning trial is: 0/30
total epoch trained: 18
Epoch 1/1
200331/200331 [======

200331/200331 [==============================] - 1s 7us/step
0.41441723247497064
0.7713643326503793
best logloss is: 0.4142774251896275
remainning trial is: 25/30
total epoch trained: 71
Epoch 1/1
200331/200331 [==============================] - 2s 8us/step
0.41442809586952517
0.7712413560749791
best logloss is: 0.4142774251896275
remainning trial is: 26/30
total epoch trained: 72
Epoch 1/1
200331/200331 [==============================] - 1s 7us/step
0.41436773525441495
0.7712323095610158
best logloss is: 0.4142774251896275
remainning trial is: 27/30
total epoch trained: 73
Epoch 1/1
200331/200331 [==============================] - 2s 8us/step
0.4144663622561178
0.7710500587273963
best logloss is: 0.4142774251896275
remainning trial is: 28/30
total epoch trained: 74
Epoch 1/1
200331/200331 [==============================] - 2s 8us/step
0.41448053389183753
0.7711874654586457
best logloss is: 0.4142774251896275
remainning trial is: 29/30
total epoch trained: 75
Epoch 1/1
200331/200331 [=

200330/200330 [==============================] - 2s 8us/step
0.4175501368865517
0.7659053727574071
best logloss is: 0.4174119570863318
remainning trial is: 2/30
total epoch trained: 22
Epoch 1/1
200330/200330 [==============================] - 1s 7us/step
0.4174602231956523
0.7664311525471723
best logloss is: 0.4174119570863318
remainning trial is: 3/30
total epoch trained: 23
Epoch 1/1
200330/200330 [==============================] - 1s 7us/step
0.4174182587861365
0.7663216990913371
best logloss is: 0.4174119570863318
remainning trial is: 4/30
total epoch trained: 24
Epoch 1/1
200330/200330 [==============================] - 1s 7us/step
0.4173907466492124
0.7664986580608283
best logloss is: 0.4173907466492124
remainning trial is: 0/30
total epoch trained: 25
Epoch 1/1
200330/200330 [==============================] - 1s 7us/step
0.417726277953501
0.7658409204710379
best logloss is: 0.4173907466492124
remainning trial is: 1/30
total epoch trained: 26
Epoch 1/1
200330/200330 [===========

200330/200330 [==============================] - 2s 8us/step
0.41741656271347244
0.7673574727895117
best logloss is: 0.4165504396230126
remainning trial is: 2/30
total epoch trained: 79
Epoch 1/1
200330/200330 [==============================] - 1s 7us/step
0.4169175087458829
0.7673369127111341
best logloss is: 0.4165504396230126
remainning trial is: 3/30
total epoch trained: 80
Epoch 1/1
200330/200330 [==============================] - 1s 7us/step
0.4173823001540481
0.7673090525506057
best logloss is: 0.4165504396230126
remainning trial is: 4/30
total epoch trained: 81
Epoch 1/1
200330/200330 [==============================] - 2s 8us/step
0.41714037762121936
0.767213939778222
best logloss is: 0.4165504396230126
remainning trial is: 5/30
total epoch trained: 82
Epoch 1/1
200330/200330 [==============================] - 1s 7us/step
0.41694515883795374
0.767096268697777
best logloss is: 0.4165504396230126
remainning trial is: 6/30
total epoch trained: 83
Epoch 1/1
200330/200330 [=========

200329/200329 [==============================] - 2s 8us/step
0.41661559951127825
0.7677406351624882
best logloss is: 0.4162512053652509
remainning trial is: 3/30
total epoch trained: 27
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.4167735940163188
0.7682082533481176
best logloss is: 0.4162512053652509
remainning trial is: 4/30
total epoch trained: 28
Epoch 1/1
200329/200329 [==============================] - 1s 7us/step
0.4170187446697202
0.7678808098936485
best logloss is: 0.4162512053652509
remainning trial is: 5/30
total epoch trained: 29
Epoch 1/1
200329/200329 [==============================] - 1s 7us/step
0.41610702332922267
0.7684144592005281
best logloss is: 0.41610702332922267
remainning trial is: 0/30
total epoch trained: 30
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.4164401327769177
0.7685611452733315
best logloss is: 0.41610702332922267
remainning trial is: 1/30
total epoch trained: 31
Epoch 1/1
200329/200329 [======

200329/200329 [==============================] - 1s 7us/step
0.4181142770519671
0.766801968030443
best logloss is: 0.41756569461097015
remainning trial is: 3/30
total epoch trained: 11
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.4175001852568679
0.766771723559287
best logloss is: 0.4175001852568679
remainning trial is: 0/30
total epoch trained: 12
Epoch 1/1
200329/200329 [==============================] - 2s 7us/step
0.41718989759843594
0.7668398317151078
best logloss is: 0.41718989759843594
remainning trial is: 0/30
total epoch trained: 13
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.41887480476144223
0.766720679606451
best logloss is: 0.41718989759843594
remainning trial is: 1/30
total epoch trained: 14
Epoch 1/1
200329/200329 [==============================] - 1s 7us/step
0.417280679189909
0.767081683315422
best logloss is: 0.41718989759843594
remainning trial is: 2/30
total epoch trained: 15
Epoch 1/1
200329/200329 [=========

200329/200329 [==============================] - 2s 8us/step
0.4162051516903617
0.7684561610796625
best logloss is: 0.41586879334750876
remainning trial is: 17/30
total epoch trained: 68
Epoch 1/1
200329/200329 [==============================] - 1s 7us/step
0.4160474565218613
0.768445196633545
best logloss is: 0.41586879334750876
remainning trial is: 18/30
total epoch trained: 69
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.41640544672192653
0.7683666379055768
best logloss is: 0.41586879334750876
remainning trial is: 19/30
total epoch trained: 70
Epoch 1/1
200329/200329 [==============================] - 2s 8us/step
0.4164332807182832
0.7688108839501293
best logloss is: 0.41586879334750876
remainning trial is: 20/30
total epoch trained: 71
Epoch 1/1
200329/200329 [==============================] - 1s 7us/step
0.41606764646901906
0.7687412816358369
best logloss is: 0.41586879334750876
remainning trial is: 21/30
total epoch trained: 72
Epoch 1/1
200329/200329 

In [19]:
def save_oof(train_df,test_df,cv,ta,ho=None,file_name='',path='../../data/nn_ebd/'):
    try:
        report = pd.read_csv(path+'report.csv')
    except:
        print('no report found! generate a new one!')
        report = pd.DataFrame()
    new_record = pd.DataFrame({'ho':[ho],'cv':[cv],'train_mean':[ta],'file':[file_name]})
    report = pd.concat([report,new_record],sort=False)
    train_df.to_pickle(path+'train/'+file_name+'.pkl')
    print(train_df.shape)
    test_df.to_csv(path+'test/'+file_name+'.csv',index=False)
    print(test_df.shape)
    report.to_csv(path+'report.csv',index=False)
    print('done!')
    

In [20]:
save_oof(train_save,test_save,cv_,ta_,ho_,file_name='oldModel_RNNModelTag_ebdNANExclude_TagNAExclude_noholdout')

(1001650, 2)
(40024, 2)
done!


In [21]:
name_dict = {'new_modelMakeOsOSV_rnnEBD':
             [['model','make','os','osv']],
             'new_modelMakeOsOSV+modelAppidInnerslotCreateWCreativeH_rnnEBD':
             [['model','make','os','osv'],['model','app_id','inner_slot_id','creative_width', 'creative_height']],
             'new_modelOSV':[['model', 'osv']],'new_model_onlyuTagRNN_100patience':[],}
